***
Name:        Find MERIT coordinates
Purpose:     uses upstream area of MERIT (UPA) and GRDC station data
             to check and correct station location

Author:      PB

Created:     15/05/2022
Copyright:   (c) PB 2022

input:  grdc_2022_10577.txt   10577 station datasets >= 10km2 upstream area or no area provided
output: grdc_MERIT_1.txt: station with new location fitted to merit UPA

No: Number from 1 ...
GRDC_No: GRDC number
lat: original latitude from GRDC metafile
lon: original longituted from GRDC metafile
newlat: corrected latitude based on MERIT UPA dataset
newlon: corrected longitute based on MERIT UPA dataset
area; provided basin area from GRDC metafile
newarea: basin area based on MERIT UPA dataset
UPS_Indicator:  min error in % from MERIT UPA to provided basin area
dist_Indicator: distance to original pour point in [unit:100m]
Indicator:  ranking criteria: UPS_Indicator + 2 x dist_indicator

***

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
#import rasterio
import rioxarray
from tqdm.auto import tqdm
from typing import Tuple

import warnings
warnings.filterwarnings('ignore')

In [2]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(message)s')
logger = logging.getLogger(__name__)

In [3]:
def find_pixel(
    upstream: xr.DataArray,
    lat: int,
    lon: int,
    area: float,
    rangexy: int = 55,
    penalty: int = 500,
    factor: int = 2,
    distance_scaler: float = .92,
    error_threshold: int = 50
) -> Tuple:
    """
    Find the coordinates of the pixel in the upstream map with a smaller error compared with a reference area.
    
    Parameters:
    -----------
    upstream: xr.DataArray
        The upstream data containing latitude and longitude coordinates.
    lat: float
        The original latitude value.
    lon: float
        The original longitude value.
    area: float
        The reference area to calculate percent error.
    rangexy: int, optional
        The range in both x and y directions to search for the new location.
    penalty: int, optional
        The penalty value to add to the distance when the percent error is too high.
    error_threshold: float, optional
        The threshold for the percent error to apply the penalty.
    factor: int, optional
        The factor to multiply with the distance for the error calculation.
    distance_scaler: float, optional
        The scaling factor for the distance calculation in pixels.
    
    Returns:
    --------
    lat_new : float
        The latitude of the new location.
    lon_new : float
        The longitude of the new location.
    min_error : float
        The minimum error value at the new location.
    """

    # find coordinates of the nearest pixel in the map
    nearest_pixel = upstream.sel(lat=lat, lon=lon, method='nearest')
    lat_orig, lon_orig = [nearest_pixel[coord].item() for coord in ['lat', 'lon']]

    # extract subset of the upstream map
    cellsize = np.mean(np.diff(upstream.lon.data))
    delta = rangexy * cellsize + 1e-6
    upstream_sel = upstream.sel(lat=slice(lat_orig + delta, lat_orig - delta),
                                lon=slice(lon_orig - delta, lon_orig + delta))

    # percent error in catchment area
    error = 100 * (1 - upstream_sel / area)

    # distance from the original pixel (in pixels)
    i = np.arange(-rangexy, rangexy + 1)
    ii, jj = np.meshgrid(i, i)
    distance = xr.DataArray(data=np.sqrt(ii**2 + jj**2) * distance_scaler, coords=upstream_sel.coords, dims=upstream_sel.dims)
    # penalise if error is too big
    distance = distance.where(error <= error_threshold, distance + penalty)

    # update error based on distance
    error += factor * distance

    # coordinates of the new location
    min_error = error.where(error == error.min(), drop=True)
    lat_new, lon_new = [min_error[coord].item() for coord in ['lat', 'lon']]
    
    return lat_new, lon_new, min_error.item()

## Configuration

In [4]:
#----------------------------------------------
# INPUT

STATION_FILE = 'stations.csv' # table of original coordinates (lat and lon) and catchment area in km2
UPSTREAM_FILE = "../data/ups_danube_3sec.tif" # MERIT Yamazaki et al 2019 - upstream area in km2

# OUTPUT
OUTPUT_FILE = 'stations_Merit_xarray.csv'

## Read input data

In [5]:
# read stations text file
try:
    stations = pd.read_csv(STATION_FILE, index_col='ID')
    new_cols = [f'{col}_new' for col in stations.columns]
    stations[new_cols] = np.nan
    logger.info(f'Table of stations correctly read: {STATION_FILE}')
except FileNotFoundError:
    logger.error(f'File not found: {STATION_FILE}')
except Exception as e:
    logger.error(f'An error occurred while reading {STATION_FILE}: {e}')
    
# read upstream map
try:
    upstream = rioxarray.open_rasterio(UPSTREAM_FILE).squeeze(dim='band')
    upstream = upstream.rename({'x': 'lon', 'y': 'lat'})
    logger.info(f'Map or upstream area corretly read: {STATION_FILE}')
except FileNotFoundError:
    logger.error(f'File not found: {UPSTREAM_FILE}')
except Exception as e:
    logger.error(f'An error occurred while reading {UPSTREAM_FILE}: {e}') 

2024-08-12 13:09:25,571 - INFO - Table of stations read: stations.csv
2024-08-12 13:09:25,587 - INFO - Map or upstream area read: stations.csv


In [ ]:
# extract cell size, top and left coordinates
#cellsize = np.mean(np.diff(upstream.lon.data))
#top = round(np.max(upstream.lat.data), 3)
#left = round(np.min(upstream.lon.data), 3)

#print(cellsize, top, left)

In [ ]:
# -----
# load upstream
#print ("read upstream")
#src = rasterio.open(UPSTREAM_FILE, "r")
#upstream = src.read(1)
#transform = src.transform
#top = round(transform[5], 3)
#left = round(transform[2],3)
#latlon = src.crs.to_epsg()
#crs = src.crs
#src.close()
#print ("done read upstream")

In [ ]:
#rows, cols = upstream.shape
#cols, rows = np.meshgrid(np.arange(cols), np.arange(rows))
# Convert pixel row/column index (row, col) to spatial coordinates (x, y)
#lon, lat = rasterio.transform.xy(transform, rows, cols)

## Processing

In [6]:
# -----------------------------------
for ID, attrs in tqdm(stations.iterrows(), total=stations.shape[0], desc='stations'):  

    # reference coordinates and upstream area
    lat_ref, lon_ref, area_ref = attrs[['lat', 'lon', 'area']]

    # search range in cells: 55 = around 5km
    rangexy = 55
    logger.debug(f'Set range to {rangexy}')
    lat, lon, error = find_pixel(upstream, lat_ref, lon_ref, area_ref, rangexy=rangexy, penalty=500, factor=2)
    
    #------------------------------------------------------
    # if still big error, increase range
    if error > 50:
        rangexy = 101
        logger.debug(f'Increase range to {rangexy}')
        lat, lon, error = find_pixel(upstream, lat_ref, lon_ref, area_ref, rangexy=rangexy, penalty=500, factor=0.5)

    # ------------------------------------------------------
    # if still big error increase range
        if error > 80:
            rangexy = 151
            logger.debug(f'Increase range to {rangexy}')
            lat, lon, error = find_pixel(upstream, lat_ref, lon_ref, area_ref, rangexy=rangexy, penalty=1000, factor=0.25)

    # -------------------------------------------------
    # find new coordinates and its associated upstream area
    stations.loc[ID, ['lat_new', 'lon_new', 'area_new']] = [round(lat, 6), round(lon, 6), int(upstream.sel(lat=lat, lon=lon).item())]

# export results
stations.sort_index(axis=1, inplace=True)
stations.to_csv(OUTPUT_FILE)
logger.info(f'Results have been exported to: {OUTPUT_FILE}')

stations:   0%|          | 0/45 [00:00<?, ?it/s]

2024-08-12 13:10:11,859 - INFO - Results have been exported to: stations_Merit_xarray.csv


In [ ]:
upstream.sel(lat=lat, lon=lon).item()

In [ ]:
def find_pixel(
    upstream: np.ndarray,
    j_orig: int,
    i_orig: int,
    upstream_ref: int,
    rangexy: int = 55,
    penalty: int = 500,
    factor: int = 2
) -> Tuple:
    """
    
    
    Parameters:
    -----------
    
    Returns:
    --------
    """
    
    area_ref = upstream[i_orig, j_orig]
    
    dim = rangexy * 2 + 1
    ind = np.zeros((dim, dim)) # indices?
    upsind = np.zeros_like(ind) # upstream index?
    diffind = np.zeros_like(ind) # distance in terms of indeces

    colcol = np.arange(j_orig - rangexy, j_orig + rangexy + 1)
    rowrow = np.arange(i_orig - rangexy, i_orig + rangexy + 1)
    
    for j, y in enumerate(rowrow):
        for i, x in enumerate(colcol):
            upsind[j, i] = 100 * np.abs(1 - upstream[y, x] / upstream_ref)
            diffind[j, i] = np.sqrt((rangexy - i)**2 + (rangexy - j)**2) * 0.92
            if upsind[j, i] > 50:
                diffind[j, i] += penalty
            ind[j, i] = upsind[j, i] + factor * diffind[j,i]

    minxy = np.where(ind == np.min(ind))
    y = minxy[0][0]
    x = minxy[1][0]
    j = rowrow[y]
    i = colcol[x]
    
    return x, y, i, j, ind[x, y]

In [ ]:
    area_orig = upstream.sel({'lat': lat_ref, 'lon': lon_ref}, method='nearest').data
    area_orig

In [ ]:
1200

In [ ]:
    # middle of the 3 sec cell
    x_center = j_orig / 1200 + left + 1 / 2400
    y_center = top - i_orig / 1200 - 1 / 2400

In [ ]:
    # search range in cells: 55 = around 5km
    rangexy = 55
    print(f'Set range to {rangexy}')
    x, y, i, j, ind = find_pixel(upstream, j_orig, i_orig, area_ref, rangexy=55, penalty=500, p=2)
    
    #------------------------------------------------------
    # if still big error, increase range
    if ind > 50:
        rangexy = 101
        print(f'Increase range to {rangexy}')
        x, y, i, j, ind = find_pixel(upstream, j_orig, i_orig, upsreal, rangexy=rangexy, penalty=500, p=0.5)


    # ------------------------------------------------------
    # if still big error increase range
        if ind > 80:
            rangexy = 151
            print(f'Increase range to {rangexy}')
            x, y, i, j, ind = find_pixel(upstream, j_orig, i_orig, upsreal, rangexy=rangexy, penalty=1000, p=0.25)

    # -------------------------------------------------
    # find new coordinates and its associated upstream area
    attrs['new_lat'] = y_center + (rangexy - y) * cellsize
    attrs['new_lon'] = x_center - (rangexy - x) * cellsize
    attrs['new_area'] = upstream[j, i]

In [ ]:
# -----------------------------------
for ID, attrs in tqdm(stations.iterrows(), total=stations.shape[0], desc='stations'):
    
    # reference coordinates and upstream area
    lat_ref, lon_ref, area_ref = attrs[['lat', 'lon', 'area']]

    # nrows, ncols = upstream.shape
    
    # find location of original coordinates, and associated upstream area
    j_orig = int((lon - left) / cellsize)
    i_orig = int((top - lat) / cellsize)
    area_orig = upstream[i_orig, j_orig]

    # middle of the 3 sec cell
    x_center = j_orig / 1200 + left + 1 / 2400
    y_center = top - i_orig / 1200 - 1 / 2400
    
    # search range in cells: 55 = around 5km
    rangexy = 55
    print(f'Set range to {rangexy}')
    x, y, i, j, ind = find_pixel(upstream, j_orig, i_orig, area_ref, rangexy=55, penalty=500, p=2)
    
    #------------------------------------------------------
    # if still big error increase range
    if ind > 50:
        rangexy = 101
        print(f'Increase range to {rangexy}')
        x, y, i, j, ind = find_pixel(upstream, j_orig, i_orig, upsreal, rangexy=rangexy, penalty=500, p=0.5)


    # ------------------------------------------------------
    # if still big error increase range
        if ind > 80:
            rangexy = 151
            print(f'Increase range to {rangexy}')
            x, y, i, j, ind = find_pixel(upstream, j_orig, i_orig, upsreal, rangexy=rangexy, penalty=1000, p=0.25)

    # -------------------------------------------------
    # find new coordinates and its associated upstream area
    attrs['new_lat'] = y_center + (rangexy - y) * cellsize
    attrs['new_lon'] = x_center - (rangexy - x) * cellsize
    attrs['new_area'] = upstream[j, i]

In [ ]:
# export results
stations.to_csv(OUTPUT_FILE)

In [ ]:
attrs[7]

In [ ]:
ID

In [ ]:
    attrs[['newlat', 'newlon', 'newarea']] = yy, xx, ups2

In [ ]:
attrs

In [ ]:
    #-------------------------------------------------
    s = str(ID)  + "\t" + attrs[7] + "\t" + attrs[8] + "\t"
    s = s + f"{yy:.5f}" + "\t" + f"{xx:.5f}" + "\t" + f"{ups2:.0f}"+ "\t"
    s = s + f"{lat:.5f}"+ "\t" +f"{lon:.5f}" + "\t"+f"{upsreal:.0f}"
    print (s)

In [ ]:





print ('done')

In [ ]:
# -----------------------------------
for stationNo in range(0, len(glofas)):

    station = glofas[stationNo].split("\t")
    upsreal = float(station[6])
    # upstream area from provider

    coord = [float(station[4]), float(station[5])]
    # lat lon
    glofas_no =  station[1]


    top = round(transform[5],3)

    left = round(transform[2],3)
    col = ups.shape[1]
    row = ups.shape[0]

    j_orig = int((coord[1] - left) * invcell)
    i_orig = int((top -coord[0]) * invcell)
    ups1 = ups[i_orig,j_orig]

    # middle of the 3 sec cell
    x_center = j_orig / 1200 + left + 1 / 2400
    y_center = top - i_orig / 1200 - 1 / 2400

    rangexy = 55
    upsups = np.zeros((rangexy*2+1,rangexy*2+1))
    ind = np.zeros((rangexy*2+1,rangexy*2+1))
    upsind = np.zeros((rangexy*2+1,rangexy*2+1))
    diffind = np.zeros((rangexy*2+1,rangexy*2+1))

    colcol = np.arange(j_orig-rangexy,j_orig+rangexy+1)
    rowrow = np.arange(i_orig-rangexy,i_orig+rangexy+1)

    j =0
    for y in rowrow:
        i = 0
        for x in colcol:
            upsind[j, i] = 100 * np.abs(1 - ups[y, x] / upsreal)
            upsups[j,i]= ups[y,x]
            diff = np.sqrt((rangexy-i)**2+(rangexy-j)**2)*0.9
            diffind[j,i] = np.sqrt((rangexy - i) ** 2 + (rangexy - j) ** 2) * 0.92
            # if upsind> 50 diff gets a penalty
            if upsind[j, i]>50:
                diffind[j, i] = diffind[j, i] + 500
            ind[j,i] = upsind[j, i] + 2 * diffind[j,i]

            i = i +1
        j = j + 1

    minxy = np.where(ind==np.min(ind))
    y=minxy[0][0]
    x=minxy[1][0]
    j = rowrow[y]
    i = colcol[x]

    #------------------------------------------------------
    # if still big error increase range
    if ind[y,x] > 50:
        print ("increase range")
        rangexy = 101
        upsups = np.zeros((rangexy*2+1,rangexy*2+1))
        ind = np.zeros((rangexy*2+1,rangexy*2+1))
        upsind = np.zeros((rangexy*2+1,rangexy*2+1))
        diffind = np.zeros((rangexy*2+1,rangexy*2+1))

        colcol = np.arange(j_orig-rangexy,j_orig+rangexy+1)
        rowrow = np.arange(i_orig-rangexy,i_orig+rangexy+1)

        j =0
        for y in rowrow:
            i = 0
            for x in colcol:
                upsind[j, i] = 100 * np.abs(1 - ups[y, x] / upsreal)
                upsups[j,i]= ups[y,x]
                diff = np.sqrt((rangexy-i)**2+(rangexy-j)**2)*0.9
                diffind[j,i] = np.sqrt((rangexy - i) ** 2 + (rangexy - j) ** 2) * 0.92
                # if upsind> 50 diff gets a penalty
                if upsind[j, i] > 50:
                    diffind[j, i] = diffind[j, i] + 500
                ind[j,i] = upsind[j, i] + 0.5 * diffind[j,i]

                i = i +1
            j = j + 1

        minxy = np.where(ind==np.min(ind))
        y=minxy[0][0]
        x=minxy[1][0]
        j = rowrow[y]
        i = colcol[x]

    #-------------------------------------------------

    # ------------------------------------------------------
    # if still big error increase range
        if ind[y, x] > 80:
            print("increase range2")
            rangexy = 151
            upsups = np.zeros((rangexy * 2 + 1, rangexy * 2 + 1))
            ind = np.zeros((rangexy * 2 + 1, rangexy * 2 + 1))
            upsind = np.zeros((rangexy * 2 + 1, rangexy * 2 + 1))
            diffind = np.zeros((rangexy * 2 + 1, rangexy * 2 + 1))

            colcol = np.arange(j_orig - rangexy, j_orig + rangexy + 1)
            rowrow = np.arange(i_orig - rangexy, i_orig + rangexy + 1)

            j = 0
            for y in rowrow:
                i = 0
                for x in colcol:
                    upsind[j, i] = 100 * np.abs(1 - ups[y, x] / upsreal)
                    upsups[j, i] = ups[y, x]
                    diff = np.sqrt((rangexy - i) ** 2 + (rangexy - j) ** 2) * 0.9
                    diffind[j, i] = np.sqrt((rangexy - i) ** 2 + (rangexy - j) ** 2) * 0.92
                    # if upsind> 50 diff gets a penalty
                    if upsind[j, i] > 50:
                        diffind[j, i] = diffind[j, i] + 1000
                    ind[j, i] = upsind[j, i] + 0.25 * diffind[j, i]

                    i = i + 1
                j = j + 1

            minxy = np.where(ind == np.min(ind))
            y = minxy[0][0]
            x = minxy[1][0]
            j = rowrow[y]
            i = colcol[x]

    # -------------------------------------------------

    yy = y_center + (rangexy - y) * cell
    xx = x_center - (rangexy - x) * cell
    ups2 = ups[j, i]    #-------------------------------------------------
    s = str(stationNo)  + "\t"+ str(glofas_no) + "\t" + station[7] + "\t" + station[8] + "\t"
    s = s + f"{yy:.5f}" + "\t" + f"{xx:.5f}" + "\t" + f"{ups2:.0f}"+ "\t"
    s = s + f"{coord[0]:.5f}"+ "\t" +f"{coord[1]:.5f}" + "\t"+f"{upsreal:.0f}"
    print (s)

    #header += "\tnewlat\tnewlon\tnewarea"
    s = glofas[stationNo].rstrip()
    s = s + "\t" + f"{yy:.6f}" + "\t" + f"{xx:.6f}" + "\t" + f"{ups2:.0f}"+ "\n"
    #s = s + "\t" + str(upsind[y,x]) +"\t"+str(diffind[y,x])+"\t"+str(ind[y,x]) + "\n"
    f = open(OUTPUT_FILE, "a")
    f.write(s)
    f.close()




print ('done')